## Excel Automation with Python

#### Agenda 

- Automating Excel with openpyxl
- Automating Excel with xlwings
- Converting VBA Code to Python Scripts
- Creating Python-based Excel Plugins/Add-ins
- Creating a macro-enabled Excel file using xlwings
- Creating Custom Excel Plugins to Execute Python Scripts

#### Comparison between `openpyxl` and `xlwings`

|    Feature                         | `openpyxl`                                   | `xlwings`                                        |
| ---------------------------------- | -------------------------------------------- | ------------------------------------------------ |
| **1. Excel Installation Required** |  No (pure Python)                            |  Yes (requires Excel installed)                  |
| **2. Real-time Excel Interaction** |  File-based only                             |  Controls open Excel app (like VBA)              |
| **3. Macro & UDF Support**         |  Cannot run macros or define UDFs            |  Can run macros and define Python-based UDFs     |
| **4. Performance**                 |  Faster for large-scale, headless automation |  Slower due to Excel COM interface               |
| **5. Charting & Visual Updates**   |  Supports static chart creation              |  Live chart creation, UI updates, event handling |

* Use **`openpyxl`** for lightweight, cross-platform, headless tasks like report generation or formatting.
* Use **`xlwings`** when you need **live Excel control**, **macros**, or **tight Excel-Python integration** like VBA replacements.

### Automating Excel with openpyxl

#### What is openpyxl?
openpyxl is a powerful Python library that allows you to read, write, and modify Excel files (.xlsx, not .xls) without needing Microsoft Excel installed.

**It's especially useful for:**

- Automating reports
- Modifying templates
- Reading structured data
- Applying formatting, formulas, charts, etc.

In [ ]:
pip install openpyxl

In [2]:
import os 
os.chdir(r"C:\Users\vaide\OneDrive - knowledgecorner.in\Course Material\Clients\Virtua Search\Excel Python Integration\Files")

###### Ex. Create a New Workbook

In [ ]:
from openpyxl import Workbook

wb = Workbook()
ws = wb.active  # Get the default sheet
ws.title = "MySheet"

ws['A1'] = "Hello"
ws['B1'] = 123

wb.save("myfile.xlsx")

###### Ex. Load Existing Workbook

In [ ]:
from openpyxl import load_workbook

wb = load_workbook("myfile.xlsx")
ws = wb.active  # Or wb['SheetName']

###### Ex. Read/Write Cell Values

In [ ]:
# Reading
value = ws['A1'].value
print(value)

# Writing
ws['A2'] = "New Value"


###### Ex. Loop Through Rows/Columns

In [ ]:
# Iterate through column and rows
for column in ws.columns:
    for row in column:
        print(row.value)

In [ ]:
# Iterate through rows and columns
for row in ws.rows:
    for col in row:
        print(col.value)

###### Ex. Add New Sheet or Access Existing Sheet

In [ ]:
# Create new sheet
ws2 = wb.create_sheet(title="Summary")

# Access existing
ws_existing = wb["MySheet"]

wb.save("myfile.xlsx")

###### Ex. Automate an Excel sheet using Python to enhance a dataset with calculated columns

- Adds 4 new columns: Rating, Bonus, Tax, Net Salary.
- Applies data validation to Rating (allowed values: 1–5).
- Calculates Bonus as a percentage of Salary based on Rating.
- Calculates Tax as (Salary + Bonus) * 10%.
- Calculates Net Salary as Salary + Bonus - Tax.

In [ ]:
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
from openpyxl.styles import Protection

wb = load_workbook("input.xlsx")
ws = wb.active

new_cols = ["Rating", "Bonus", "Tax", "Net Salary"]
existing_headers = [cell.value for cell in ws[1]]

# Add new columns only if not present
for col_name in new_cols:
    if col_name not in existing_headers:
        ws.cell(row=1, column=ws.max_column + 1, value=col_name)

# Refresh header map and prepare col_letter lookup
headers = [cell.value for cell in ws[1]]
col_letters = {name: get_column_letter(idx + 1) for idx, name in enumerate(headers)}

# Apply data validation for "Rating"
rating_col = col_letters["Rating"]
rating_range = f"{rating_col}2:{rating_col}{ws.max_row}"
dv = DataValidation(type="list", formula1='"Excellent,Good,Average,Below Average,Poor"', allow_blank=False)
dv.prompt = "Choose a performance rating"
dv.error = "Select from the list"
ws.add_data_validation(dv)
dv.add(rating_range)

# Add formulas row-wise
for row in range(2, ws.max_row + 1):
    salary = f"{col_letters['Salary']}{row}"
    rating = f"{col_letters['Rating']}{row}"
    bonus = f"{col_letters['Bonus']}{row}"
    tax = f"{col_letters['Tax']}{row}"
    net = f"{col_letters['Net Salary']}{row}"

    ws[bonus] =  f'=IF({rating}="Excellent", {salary} * 0.12, IF({rating}="Good", {salary} * 0.1, IF({rating}="Average", {salary} * 0.07, IF({rating}="Below Average", {salary} * 0.05, {salary} * 0.01))))'
    ws[tax] = f"=({salary}+{bonus})*0.1"
    ws[net] = f"={salary}+{bonus}-{tax}"

Hide intermediate columns
for col in ["Bonus", "Tax"]:
    ws.column_dimensions[col_letters[col]].hidden = True

# Lock all cells except Rating column (hide formulas)
rating_col_idx = headers.index("Rating") + 1
for row in ws.iter_rows(min_row=2):
    for cell in row:
        if cell.column == rating_col_idx:
            cell.protection = Protection(locked=False, hidden=True)
        else:
            cell.protection = Protection(locked=True, hidden=True)

# Protect worksheet
ws.protection.enable()
ws.protection.set_password("yourpassword")

wb.save("input.xlsx")


###### Ex. Copy Excel file
Copy data from one excel to another maintaining the formatting

In [ ]:
import shutil
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.formatting.rule import FormulaRule

# Step 1: Copy source file to destination (preserves everything)
shutil.copyfile("input.xlsx", "destination.xlsx")

# Step 2: Open the destination file
wb = load_workbook("destination.xlsx")
ws = wb.active  # or wb['SheetName']

# Step 3: Define a conditional formatting rule
# Example: Highlight rows where column D = "Poor" (adjust column as needed)
fill = PatternFill(start_color="FFC7CE", end_color="FFC7CE", fill_type="solid")

# Apply to each row (A to E shown; adjust as per your sheet)
for row in range(2, ws.max_row + 1):
    formula = f'=$D{row}="Poor"'  # Change column D to your target column
    ws.conditional_formatting.add(f"A{row}:{get_column_letter(ws.max_column)}{row}", FormulaRule(formula=[formula], fill=fill))

# Step 4: Save changes
wb.save("destination.xlsx")


### Automating Excel with xlwings

#### What is xlwings?

xlwings is a Python library that enables automated interaction with Excel using the Excel COM API. Unlike openpyxl, it requires Microsoft Excel installed and allows live interaction with open workbooks, making it great for:

- Excel automation
- User-defined functions (UDFs)
- Integration with Excel macros
- Interactive dashboards or calculators

#### Quick tools - 

Here's a handy **`xlwings` Cheatsheet** for quick reference — covering the **most commonly used operations** with examples.

**Open or Create Workbook**

```python
import xlwings as xw

wb = xw.Book("file.xlsx")      # Open existing
wb = xw.Book()                 # New workbook
wb.save("file.xlsx")          # Save as
wb.close()                    # Close workbook
```
---
**Access Sheets**

```python
ws = wb.sheets[0]             # By index
ws = wb.sheets['Sheet1']      # By name
ws = wb.sheets.add("New")     # Add new sheet
ws.delete()                   # Delete sheet
```

---

**Read/Write Data**

```python
ws.range("A1").value = "Hello"             # Write single value
value = ws.range("A1").value               # Read value

ws.range("A1:C3").value = [[1,2,3],[4,5,6]]  # Write 2D list
data = ws.range("A1").expand().value        # Read table
```

---

**Charts**

```python
chart = ws.charts.add()
chart.chart_type = 'line'
chart.set_source_data(ws.range("A1:B10"))
chart.api[1].ChartTitle.Text = "Sales Trend"
```

---

**Formatting**

```python
cell = ws.range("A1")
cell.color = (255, 255, 0)              # Background color
cell.api.Font.Bold = True              # Bold
cell.api.Font.Color = 0xFF0000         # Red font
cell.api.NumberFormat = "$#,##0.00"    # Currency format
```

---

**Looping Ranges**

```python
for cell in ws.range("A1:A5"):
    print(cell.value)
```

---

**Data Validation**

```python
ws.range("A1").api.Validation.Add(
    Type=3,  # List
    Formula1="Option1,Option2,Option3"
)
```

---

**Sheet & Cell Protection**

```python
ws.api.Protect(Password="mypassword")

cell = ws.range("A2")
cell.api.Locked = False
cell.api.FormulaHidden = True
```

---

**PasteSpecial (formats only)**

```python
ws.range("A1:D1").api.Copy()
ws.range("A2:D2").api.PasteSpecial(Paste=-4122)  # Formats only
```

---

**Used Range, Last Row/Col**

```python
ws.used_range.rows.count
ws.used_range.columns.count

last_row = ws.range("A1").end("down").row
last_col = ws.range("A1").end("right").column
```

---

**Tips**

* `xw.Book().app.visible = False` to run Excel in background
* `.formula` vs `.value`: to write Excel formulas like `=SUM(A1:A5)`
* Always `save()` before closing to persist changes

---

In [ ]:
pip install xlwings

###### Ex. Open or Connect to Excel Workbook

In [ ]:
import xlwings as xw

# Start a new instance of Excel and open a workbook
wb = xw.Book("input.xlsx")  # Opens existing file
# wb = xw.Book()  # Opens a new workbook

###### Ex. Access Sheets and Cells

In [ ]:
sheet = wb.sheets["Sheet1"]

# Read and write values
# sheet["A1"].value = "Hello, Excel"
# print(sheet["A1"].value)

###### Ex. Count number of rows and columns

In [ ]:
ws = xw.Book("input.xlsx").sheets[0]
rows = ws.used_range.rows.count
cols = ws.used_range.columns.count
rows, cols

In [ ]:
for i in ws.used_range.rows :
    print(i.value)

In [ ]:
for i in ws.used_range.columns :
    print(i.value)

###### Ex. Read/Write Ranges

In [ ]:
# Writing a list to Excel
sheet.range("A2:A5").value = ["One", "Two", "Three", "Four"]

# Reading back a range
data = sheet.range("A2:A5").value
print(data)

###### Ex. Loop and Update Multiple Cells

In [ ]:
for i in range(1, 6):
    sheet.range(f"B{i}").value = i * 10

###### Ex. Save and Close Workbook

In [ ]:
wb.save("updated_example.xlsx")
wb.close()

###### Ex. Read data from multiple sheets in excel and combine into one (append and merge)

In [ ]:
# Append
import xlwings as xw

# Open the Excel file
wb = xw.Book("monthly_data.xlsx")
output_sheet = wb.sheets.add("Combined", after=wb.sheets[-1])  # Create new sheet for combined data

row_cursor = 1  # Start from first row in output
for sheet in wb.sheets:
    if sheet.name == "Combined":
        continue  # Skip output sheet if re-running

    data = sheet.used_range.value[1:]
    if data:
        output_sheet.range(f"A{row_cursor}").value = data
        row_cursor += len(data)  # Move cursor for next sheet's data

# Optional: Save and close
wb.save("monthly_data.xlsx")
# # wb.close()


In [ ]:
# Merge
import xlwings as xw

# Open the Excel file
wb = xw.Book("Employees.xlsx")
emp_sheet = wb.sheets["Employee"]

 # Add header for new column
emp_sheet.range("C1").value = "Department"

# Insert VLOOKUP formula from row 2 onward
last_row = emp_sheet.range("A1").end("down").row
for row in range(2, last_row + 1):
    emp_sheet.range(f"C{row}").formula = (
        f'=VLOOKUP(B{row}, Salary!A:B, 2, FALSE)'
    )

wb.save()

In [ ]:
import pandas as pd

# Load all sheets into a dictionary of DataFrames
xls = pd.read_excel("monthly_data.xlsx", sheet_name=None)

# Combine all DataFrames
combined_df = pd.concat(xls.values(), ignore_index=True)

# Save to a new sheet
with pd.ExcelWriter("monthly_data.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    combined_df.to_excel(writer, sheet_name="Combined", index=False)

###### Ex. Create a Chart with Data via Python
**Objective**: Populate data and generate a column chart dynamically.

In [ ]:
import pandas as pd
import xlwings as xw

# Step 1: Load the CSV
df = pd.read_excel("employee_car.xlsx")

# Step 3: Open with xlwings
wb = xw.Book("employee_car.xlsx")
data_sheet = wb.sheets["data"]
dashboard = wb.sheets.add("Dashboard")

# Step 4: Define chart creator helper
def add_chart(chart_type, left, top, width, height, data_range, title):
    chart = dashboard.charts.add()
    chart.chart_type = chart_type
    chart.set_source_data(data_range)
    chart.title = title
    chart.left, chart.top, chart.width, chart.height = left, top, width, height

# Step 5: Insert charts (adjust based on actual data columns)
# Example chart 1: Count by Gender (Pie)
pivot = df["Gender"].value_counts()
dashboard.range("A1").options(index=False, header=False).value = pivot.reset_index().values.tolist()
chart_data = dashboard.range("A1").expand()
add_chart("pie", 10, 10, 300, 200, chart_data, "Gender Distribution")

# Example chart 3: Age Distribution (Column Chart)
age_counts = df["Age"].value_counts().sort_index()
dashboard.range("A20").value = age_counts.reset_index().values.tolist()
add_chart("column_clustered", 10, 220, 300, 200, dashboard.range("A20").expand(), "Age Distribution")

# Example chart 4: Salary by Gender (Line Chart)
salary_gender = df.groupby("Gender")["Salary"].mean()
dashboard.range("A30").value = salary_gender.reset_index().values.tolist()
add_chart("line", 350, 220, 300, 200, dashboard.range("A30").expand(), "Avg Salary by Gender")

# Save and show file
# wb.save()
# wb.app.visible = True  # Keep Excel open for viewing


###### Ex.  Run Macro to highlight employee who are managers

In [ ]:
import xlwings as xw

# Step 2: Open copied workbook
wb = xw.Book("employee_car_macro.xlsm")

# Step 3: Run macro (macro must be defined in the VBA editor in this workbook)
macro = wb.macro("HighlightManagerRowValuesOnly")
macro()

# Step 4: Save and close
wb.save()

In [5]:
wb.close()

In [6]:
import pandas as pd
df = pd.read_excel("employee_car.xlsx")
df.to_excel("Data2.xlsx", index=False)

In [7]:
import xlwings as xw

# Step 1: Launch Excel
app = xw.App(visible=True)

# Step 2: Open Data workbook (target)
data_wb = app.books.open("Data2.xlsx")

# Step 3: Open or refer to loaded Add-in
addin = app.books.open("Highlighter.xlam")  # Or use app.books["Highlighter.xlam"] if already open

# Step 4: Activate the data workbook (so macro runs on this)
data_wb.activate()

# Step 5: Call the macro from add-in
app.macro("Highlighter.xlam!Module1.HighlightManagerRowValuesOnly")()

# Optional: Save and close target file
data_wb.save()
# data_wb.close()
# app.quit()  # If you want to close Excel


###### Ex. Define a Python UDF (User Defined Function) Callable from Excel
**Objective**: Create a Python function that can be used directly in Excel cells.

📄 `my_udf.py`

In [ ]:
import xlwings as xw

@xw.func
def profit_margin(sales, cost):
    if sales == 0:
        return "N/A"
    return round((sales - cost) / sales, 2)

**How to use**:

* Save this file.
* Run: `xlwings runpython install` (once to register addin)
* Open Excel and type: `=profit_margin(1000, 750)`

###### Ex. Create a Python-Powered Excel Macro Button
**Objective**: Add a button in Excel that runs a Python script when clicked.

In [ ]:
### Step-by-step:

# 1.Create Python script (e.g., `button_macro.py`)

import xlwings as xw

def run_update():
    wb = xw.Book.caller()
    sheet = wb.sheets[0]
    sheet["A1"].value = "Updated from Button Click"

**2. Link Excel button**

   * Insert a shape or button in Excel.
   * Assign macro name: `RunPython ("import button_macro; button_macro.run_update()")`

- Now clicking the button will run your Python logic, just like a VBA macro.